### Low-resource setting: Fine-tuning data sizes are [50, 100, 200, 300, 400]

In [8]:
import pandas as pd
import os, re, copy
from glob import glob
from reports import get_average_classification_report
pd.options.display.float_format = '{:.4f}'.format
from functools import reduce
pd.options.display.float_format = '{:.4f}'.format
import seaborn as sns

In [2]:
target_trained_with_nums = [50, 100, 200, 300, 400]
dirnames = sorted(glob(os.path.join('C:\DATA\ComBERT', 'results_spamFiltering', '*_seed4')))
len(dirnames)

32

In [3]:
dfs = []
for dirname in dirnames:
    test_company, model_name, method_name, epoch_num, seed_num = os.path.basename(dirname).split('_')
    
    records = []
    for target_trained_with_num in target_trained_with_nums:
        filepaths = glob(os.path.join(os.path.dirname(dirname), '{}_*'.format('_'.join([test_company, model_name, method_name, epoch_num])), \
                  'classification_report_train_{}.csv'.format(target_trained_with_num)))
        
        column_name = '_'.join([test_company, model_name, method_name, epoch_num])
        averaged_classification_report_df = get_average_classification_report([pd.read_csv(filepath) for filepath in filepaths])
        acc = 100*averaged_classification_report_df.filter(items = ['accuracy'], axis=0)['f1-score'].values[0]
        records.append((target_trained_with_num, acc))
    dfs.append(pd.DataFrame(records, columns=['trained_with', column_name]).sort_values(by=['trained_with']))

result = reduce(lambda df1,df2: pd.merge(df1,df2,on='trained_with'), dfs)
result.set_index('trained_with', inplace=True)
result.loc['Average'] = result.mean()

result

,testCompanyName=n_Araci_CM_epoch7,testCompanyName=n_Araci_NoPT_epoch7,testCompanyName=n_Araci_SM_epoch7,testCompanyName=n_Araci_WWM_epoch7,testCompanyName=n_BERT_CM_epoch7,testCompanyName=n_BERT_NoPT_epoch7,testCompanyName=n_BERT_SM_epoch7,testCompanyName=n_BERT_WWM_epoch7,testCompanyName=n_SECBERT_CM_epoch7,testCompanyName=n_SECBERT_NoPT_epoch7,...,testCompanyName=y_BERT_SM_epoch7,testCompanyName=y_BERT_WWM_epoch7,testCompanyName=y_SECBERT_CM_epoch7,testCompanyName=y_SECBERT_NoPT_epoch7,testCompanyName=y_SECBERT_SM_epoch7,testCompanyName=y_SECBERT_WWM_epoch7,testCompanyName=y_Yang_CM_epoch7,testCompanyName=y_Yang_NoPT_epoch7,testCompanyName=y_Yang_SM_epoch7,testCompanyName=y_Yang_WWM_epoch7
trained_with,,,,,,,,,,,,,,,,,,,,,
50,53.9600,47.6800,60.9000,49.2200,48.7600,52.6800,50.3400,44.8800,58.8400,54.6200,...,49.8400,51.6000,59.4400,56.0800,59.2200,51.2200,58.8800,46.9000,56.7400,51.5200
100,63.5600,56.8800,67.2000,59.1400,61.4000,56.8400,60.8200,56.9200,69.0800,61.3000,...,60.3600,53.5400,66.0000,66.0800,69.8200,61.5800,68.5000,63.1000,67.7800,61.8600
200,74.2800,65.0600,75.8800,76.2400,71.0000,70.4800,72.7200,70.2000,76.5600,73.2600,...,77.2400,61.0800,79.1800,76.3400,75.5800,69.9200,77.6200,74.3400,79.3400,68.8000
300,77.8200,70.3800,78.3400,78.0200,77.6800,76.3200,76.9800,76.2600,78.5600,73.8000,...,78.8000,68.3800,80.7800,77.4000,77.1000,76.1800,80.4400,79.4800,81.1000,74.6800
400,76.6400,72.8800,74.9200,78.3400,77.3600,75.9600,75.9400,78.1200,79.1000,76.6600,...,79.6600,70.1200,80.6200,77.8000,78.9400,77.3400,79.9400,79.8000,79.3800,75.7000
Average,69.2520,62.5760,71.4480,68.1920,67.2400,66.4560,67.3600,65.2760,72.4280,67.9280,...,69.1800,60.9440,73.2040,70.7400,72.1320,67.2480,73.0760,68.7240,72.8680,66.5120


### Evalautaion of improved performance of each model
* Based on the average scores in the test setting using tweets containing company names, **SEC-BERT** and **Yang's FinBERT** using the proposed company name masking method achieved high spam filtering performance.
    - **yCM** > (yNoPT, ySM)

* Based on the average scores in the test setting using tweets containing *no* company names, **SEC-BERT** and **Yang's FinBERT** using the proposed company name masking method achieved *reasonably* high spam filtering performance.
    - **nCM** > (nNoPT, nSM, nWWM)

In [9]:
def show_table(model_name):
    print(model_name)
    dfs = []
    for setting in ['testCompanyName=n', 'testCompanyName=y']:
        one_model_setting_result = result[[col for col in result.columns if (model_name in col) and (setting in col)]]
        one_model_setting_result.columns = [item.replace(setting, setting[-1]).replace(model_name, '')\
                                            .replace('epoch7', '').replace('_', '')\
                                            for item in one_model_setting_result.columns]
        dfs.append(one_model_setting_result)

    one_model_result = reduce(lambda df1,df2: pd.merge(df1,df2,on='trained_with'), dfs)

    return one_model_result[sorted(one_model_result.columns)]

### SEC-BERT
* Based on the average scores in the no post-training setting, it achieved the best performance in filtering spam tweets containing company names
    - [yNoPT] **SEC-BERT (70.7400)** > BERT (70.0000) > Yang (68.7240) > Araci (63.9800)

In [12]:
show_table('SECBERT')

SECBERT


,nCM,nNoPT,nSM,nWWM,yCM,yNoPT,ySM,yWWM
trained_with,,,,,,,,
50,58.8400,54.6200,59.0200,54.4600,59.4400,56.0800,59.2200,51.2200
100,69.0800,61.3000,67.4800,63.9400,66.0000,66.0800,69.8200,61.5800
200,76.5600,73.2600,71.9000,71.5200,79.1800,76.3400,75.5800,69.9200
300,78.5600,73.8000,75.7000,73.4200,80.7800,77.4000,77.1000,76.1800
400,79.1000,76.6600,76.5200,77.7800,80.6200,77.8000,78.9400,77.3400
Average,72.4280,67.9280,70.1240,68.2240,73.2040,70.7400,72.1320,67.2480


### Yang's FinBERT
* Based on the average scores in the no post-training setting, it achieved the best performance in filtering spam tweets containing no company names
    - [nNoPT] **Yang (68.4520)** > SEC-BERT (67.9280) > BERT (66.4560) > Araci (62.5760)

In [5]:
show_table('Yang')

Yang


,nCM,nNoPT,nSM,nWWM,yCM,yNoPT,ySM,yWWM
trained_with,,,,,,,,
50,52.1200,55.4000,49.3600,53.7600,58.8800,46.9000,56.7400,51.5200
100,62.3000,61.0200,59.2000,61.6600,68.5000,63.1000,67.7800,61.8600
200,73.7800,70.8800,72.9800,70.0000,77.6200,74.3400,79.3400,68.8000
300,77.0000,77.0800,77.7800,78.1000,80.4400,79.4800,81.1000,74.6800
400,75.4400,77.8800,77.9600,75.8200,79.9400,79.8000,79.3800,75.7000
Average,68.1280,68.4520,67.4560,67.8680,73.0760,68.7240,72.8680,66.5120


In [6]:
show_table('Araci')

Araci


,nCM,nNoPT,nSM,nWWM,yCM,yNoPT,ySM,yWWM
trained_with,,,,,,,,
50,53.9600,47.6800,60.9000,49.2200,54.0800,42.3800,59.1400,48.5000
100,63.5600,56.8800,67.2000,59.1400,62.4000,62.0200,68.8400,51.2000
200,74.2800,65.0600,75.8800,76.2400,77.1800,69.2600,79.6800,62.1000
300,77.8200,70.3800,78.3400,78.0200,79.6800,71.2600,79.6400,70.1000
400,76.6400,72.8800,74.9200,78.3400,76.9400,74.9800,78.1800,72.4400
Average,69.2520,62.5760,71.4480,68.1920,70.0560,63.9800,73.0960,60.8680


In [7]:
show_table('_BERT_')

_BERT_


,nCM,nNoPT,nSM,nWWM,yCM,yNoPT,ySM,yWWM
trained_with,,,,,,,,
50,48.7600,52.6800,50.3400,44.8800,51.9000,54.5000,49.8400,51.6000
100,61.4000,56.8400,60.8200,56.9200,65.0200,63.6200,60.3600,53.5400
200,71.0000,70.4800,72.7200,70.2000,73.4200,75.8600,77.2400,61.0800
300,77.6800,76.3200,76.9800,76.2600,79.5800,78.8600,78.8000,68.3800
400,77.3600,75.9600,75.9400,78.1200,78.8800,77.1600,79.6600,70.1200
Average,67.2400,66.4560,67.3600,65.2760,69.7600,70.0000,69.1800,60.9440
